# PyTorch benchmark, profiler and JIT compiler

In this notebook we will see a simple example on how to use the PyTorch benchmark and profiler, and discuss the impact of Just-in-Time compilation on performance of custom functions.
We will use the mean squared error loss for its simplicity.

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['OMP_NUM_THREADS'] = '12' # for Numpy and BLAS
  
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.utils import data
from torch.profiler import profile, record_function, ProfilerActivity
import torch.utils.benchmark as benchmark

from tqdm.notebook import trange, tqdm

In [2]:
# Sample random scalar data
nsamples = 20000
nfeat = 500

x = np.random.random([nsamples, nfeat])
y = np.random.random([nsamples, nfeat])

In [3]:
def mse(input, target):
    '''
    measures the mean squared error (squared L2 norm) between
    each element in the input `x` and target :`y`
    '''
    diff = (input - target)  # WARN: this temporary variable makes this function 10-20% slower in Numpy
    return (diff**2).mean()

In [4]:
# Time the MSE function
%timeit -n1 -r100 mse(x, y)

76.7 ms ± 337 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


## PyTorch benchmark (CPU)

In [5]:
# Copyless Tensors
tx = torch.from_numpy(x)
ty = torch.from_numpy(y)

tx.dtype

torch.float64

In [6]:
torch.set_num_threads(12)

%timeit -n1 -r100 mse(tx, ty)

15.8 ms ± 1.01 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## GPU benchmarks

In [7]:
device = 'cuda'
cx = tx.to(device)
cy = ty.to(device)

# %timeit -n1 -r100 mse(cx, cy) ## !!! Wrong way of time GPU functions, without torch.cuda.synchronize()
## https://pytorch.org/tutorials/recipes/recipes/benchmark.html#benchmarking-with-torch-utils-benchmark-timer

In [8]:
t0 = benchmark.Timer(
    stmt='mse(x, y)',
    setup='from __main__ import mse',
    globals={'x': cx, 'y': cy})

print(t0.timeit(100))

mse(x, y)
setup: from __main__ import mse
  978.00 us
  1 measurement, 100 runs , 1 thread


In [9]:
with profile(activities=[ProfilerActivity.CUDA]) as prof:
    with record_function("mse"):
        mse(cx, cy)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
prof.export_chrome_trace("mse.trace.json")

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         0.00%       0.000us       0.000us     473.000us        48.56%     473.000us     473.000us             1  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         0.00%       0.000us       0.000us     347.000us        35.63%     347.000us     347.000us             1  
void at::

In [10]:
df = pd.DataFrame({e.key:e.__dict__ for e in prof.key_averages()}).T
df[['count', 'cpu_time_total', 'cuda_time_total']].sort_values(['cpu_time_total', 'cuda_time_total'], ascending=False)

,count,cpu_time_total,cuda_time_total
cudaLaunchKernel,3,1652050,0
cudaDeviceSynchronize,1,676,0
cudaMemsetAsync,1,11,0
"void at::native::vectorized_elementwise_kernel<4, at::native::BinaryFunctor<double, double, double, at::native::AddFunctor<double> >, at::detail::Array<char*, 3> >(int, at::native::BinaryFunctor<double, double, double, at::native::AddFunctor<double> >, at::detail::Array<char*, 3>)",1,0,473
"void at::native::vectorized_elementwise_kernel<4, at::native::(anonymous namespace)::pow_tensor_scalar_kernel_impl<double, double>(at::TensorIteratorBase&, double)::{lambda(double)#1}, at::detail::Array<char*, 2> >(int, at::native::(anonymous namespace)::pow_tensor_scalar_kernel_impl<double, double>(at::TensorIteratorBase&, double)::{lambda(double)#1}, at::detail::Array<char*, 2>)",1,0,347
"void at::native::reduce_kernel<512, 1, at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4> >(at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4>)",1,0,153
Memset (Device),1,0,1


# PyTorch Just-in-Time compiler

https://pytorch.org/docs/stable/jit.html

#### Scripting vs. Tracing

- `torch.jit.script` compiles the function or module into TorchScript, using a \[large\] subset of Python
- `torch.jit.trace` uses the example input to compute a fixed graph, and therefore cannot handle control flow, e.g. if statements and similar conditions

In [11]:
# @torch.jit.script
# def jit_mse(input, target):
#     return ((input - target)**2).mean()

# jit_mse = torch.jit.script(mse)
jit_mse = torch.jit.trace(mse, example_inputs=(cx, cy))
print(jit_mse.code)
print(jit_mse.graph)

def mse(input: Tensor,
    target: Tensor) -> Tensor:
  _0 = torch.pow(torch.sub(input, target), 2)
  return torch.mean(_0)

graph(%input : Double(20000, 500, strides=[500, 1], requires_grad=0, device=cuda:0),
      %target : Double(20000, 500, strides=[500, 1], requires_grad=0, device=cuda:0)):
  %2 : int = prim::Constant[value=1]() # /tmp/ipykernel_7031/3968975852.py:6:0
  %3 : Double(20000, 500, strides=[500, 1], requires_grad=0, device=cuda:0) = aten::sub(%input, %target, %2) # /tmp/ipykernel_7031/3968975852.py:6:0
  %4 : int = prim::Constant[value=2]() # /scratch/snx3000/dealmeih/ds/deepspeed-env/lib/python3.8/site-packages/torch/_tensor.py:30:0
  %5 : Double(20000, 500, strides=[500, 1], requires_grad=0, device=cuda:0) = aten::pow(%3, %4) # /scratch/snx3000/dealmeih/ds/deepspeed-env/lib/python3.8/site-packages/torch/_tensor.py:30:0
  %6 : NoneType = prim::Constant()
  %7 : Double(requires_grad=0, device=cuda:0) = aten::mean(%5, %6) # /tmp/ipykernel_7031/3968975852.py:7:0
  return

In [12]:
t1 = benchmark.Timer(
    stmt='jit_mse(x, y)',
    setup='from __main__ import jit_mse',
    globals={'x': cx, 'y': cy})

print(t1.timeit(100))

jit_mse(x, y)
setup: from __main__ import jit_mse
  594.62 us
  1 measurement, 100 runs , 1 thread


In [13]:
with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("jit_mse"):
        jit_mse(cx, cy)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
prof.export_chrome_trace("jit_mse.trace.json")

df = pd.DataFrame({e.key:e.__dict__ for e in prof.key_averages()}).T
df[['count', 'cpu_time_total', 'cuda_time_total']].sort_values(['cpu_time_total', 'cuda_time_total'], ascending=False)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          fused_sub_pow         0.00%       0.000us         0.00%       0.000us       0.000us     435.000us        74.11%     435.000us     435.000us             1  
void at::native::reduce_kernel<512, 1, at::native::R...         0.00%       0.000us         0.00%       0.000us       0.000us     151.000us        25.72%     151.000us     151.000us             1  
         

,count,cpu_time_total,cuda_time_total
cudaDeviceSynchronize,1,399,0
cudaLaunchKernel,1,14,0
cudaMemsetAsync,1,13,0
fused_sub_pow,1,0,435
"void at::native::reduce_kernel<512, 1, at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4> >(at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4>)",1,0,151
Memset (Device),1,0,1


## Comparing with torch.nn.functional.mse_loss

In [14]:
## F.mse_loss(cx, cy)
assert jit_mse(cx, cy) - F.mse_loss(cx, cy) < 1e-12

t2 = benchmark.Timer(
    stmt='mse_loss(x, y)',
    setup='from torch.nn.functional import mse_loss',
    globals={'x': cx, 'y': cy})

print(t2.timeit(100))

mse_loss(x, y)
setup: from torch.nn.functional import mse_loss
  631.29 us
  1 measurement, 100 runs , 1 thread


In [15]:
with profile(activities=[ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("mse_loss"):
        F.mse_loss(cx, cy)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
prof.export_chrome_trace("mse_loss.trace.json")

df = pd.DataFrame({e.key:e.__dict__ for e in prof.key_averages()}).T
df[['count', 'cpu_time_total', 'cuda_time_total']].sort_values(['cpu_time_total', 'cuda_time_total'], ascending=False)

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
void at::native::vectorized_elementwise_kernel<4, at...         0.00%       0.000us         0.00%       0.000us       0.000us     472.000us        75.40%     472.000us     472.000us             1  
void at::native::reduce_kernel<512, 1, at::native::R...         0.00%       0.000us         0.00%       0.000us       0.000us     152.000us        24.28%     152.000us     152.000us             1  
         

,count,cpu_time_total,cuda_time_total
cudaLaunchKernel,2,1195159,0
cudaDeviceSynchronize,1,444,0
cudaMemsetAsync,1,14,0
"void at::native::vectorized_elementwise_kernel<4, at::native::mse_kernel_cuda(at::TensorIterator&)::{lambda()#1}::operator()() const::{lambda()#2}::operator()() const::{lambda(double, double)#1}, at::detail::Array<char*, 3> >(int, at::native::mse_kernel_cuda(at::TensorIterator&)::{lambda()#1}::operator()() const::{lambda()#2}::operator()() const::{lambda(double, double)#1}, at::detail::Array<char*, 3>)",1,0,472
"void at::native::reduce_kernel<512, 1, at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4> >(at::native::ReduceOp<double, at::native::MeanOps<double, double>, unsigned int, double, 4>)",1,0,152
Memset (Device),1,0,2


Our simple JIT'ed function is actually slightly faster than the PyTorch mse_loss with it's custom CUDA kernel, however, the same is not true in backward pass as we can see below.

In [16]:
var = cx.clone()
var.requires_grad = True

# warm up
jit_mse(var, cy).backward()
var.grad = None

t3 = benchmark.Timer(
    stmt='jit_mse(x, y).backward()',
    setup='from __main__ import jit_mse',
    globals={'x': var, 'y': cy})

t4 = benchmark.Timer(
    stmt='mse_loss(x, y).backward()',
    setup='from torch.nn.functional import mse_loss',
    globals={'x': var, 'y': cy})

print(t3.timeit(100))
print(t4.timeit(100))

jit_mse(x, y).backward()
setup: from __main__ import jit_mse
  2.04 ms
  1 measurement, 100 runs , 1 thread
mse_loss(x, y).backward()
setup: from torch.nn.functional import mse_loss
  1.75 ms
  1 measurement, 100 runs , 1 thread
